# **PCB ERROR DETECTION AND CLASSIFICATION**

---



---

## **INFERENCE**

---

In [ ]:
!pip install imagehash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 15.9 MB/s eta 0:00:00


# PCB Differential Detection Pipeline

In [ ]:
import os
import time
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from torchvision.ops import nms
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import imagehash
from skimage.metrics import structural_similarity as ssim
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


LOAD MODEL

---



In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/best_resnet50_pcb_defects_50epochs_patches_1.pth"

# INSPECT CHECKPOINT FIRST
checkpoint = torch.load(model_path, map_location=device)
print("🔍 Checkpoint keys:", list(checkpoint.keys()))

# EXTRACT CLASS NAMES
class_names = checkpoint.get("class_names", None)
if class_names is None:
    print("⚠️ No 'class_names' in checkpoint")
    # Standard DeepPCB order
    class_names = [
        'missing_hole', 'mouse_bite', 'open_circuit',
        'short', 'spur', 'spurious_copper'
    ]
    print("✅ Using standard DeepPCB class order")

# Remove 'normal' if present
if 'normal' in class_names:
    class_names.remove('normal')

num_classes = len(class_names)
print(f"✅ {num_classes} classes: {class_names}")

# Create & load model
defect_classifier = models.resnet50(weights=None)
in_features = defect_classifier.fc.in_features
defect_classifier.fc = torch.nn.Linear(in_features, num_classes)

defect_classifier.load_state_dict(checkpoint["model_state_dict"])
defect_classifier.to(device)
defect_classifier.eval()

# VERIFY: Model output matches class count
dummy_input = torch.randn(1, 3, 224, 224).to(device)
with torch.no_grad():
    dummy_out = defect_classifier(dummy_input)
    print(f"✅ Model output shape: {dummy_out.shape} == [1, {num_classes}] ✓")


# CONFIG

golden_images_dir = "/content/drive/MyDrive/PCB_DATASET/PCB_USED/"
WINDOW_SIZE = 96
STRIDE = 32
SIMILARITY_THRESHOLD = 0.97
CLASSIFIER_CONFIDENCE_THRESHOLD = 0.90

inference_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# GOLDEN DATABASE

def create_golden_image_database(golden_dir):
    db = []
    filenames = os.listdir(golden_dir)
    for fname in filenames:
        path = os.path.join(golden_dir, fname)
        try:
            img = Image.open(path).convert('RGB')
            hash_val = imagehash.phash(img)
            db.append({'filename': fname, 'image': img, 'hash': hash_val})
        except Exception as e:
            print(f"Warning: Could not load '{path}'. Error: {e}")
    return db

golden_db = create_golden_image_database(golden_images_dir)
print(f"✅ {len(golden_db)} golden images loaded")


# DEBUG FUNCTION

def debug_top_predictions(patch_tensor, window_coords):
    """Print top-3 predictions for debugging wrong labels"""
    with torch.no_grad():
        outputs = defect_classifier(patch_tensor)
        probabilities = F.softmax(outputs, dim=1)
        top_probs, top_indices = torch.topk(probabilities, 3)


    print(f"🔍 Window {window_coords}: Top-3 predictions:")
    for i in range(3):
        idx = top_indices[0][i].item()
        prob = top_probs[0][i].item()
        print(f"  {i+1}. {class_names[idx]}: {prob:.3f}")

    return top_indices[0][0].item(), top_probs[0][0].item()

# CORE FUNCTIONS (FIXED CLASS MAPPING)

def find_best_match(input_image, golden_database):
    if not golden_database:
        return None
    input_hash = imagehash.phash(input_image)
    best_match = min(golden_database, key=lambda x: input_hash - x['hash'])
    return best_match['image']

def detect_anomalies_by_comparison(input_image, golden_image, classifier):
    if input_image.size != golden_image.size:
        golden_image = golden_image.resize(input_image.size)

    detections = []
    img_width, img_height = input_image.size
    debug_count = 0

    for y in range(0, img_height - WINDOW_SIZE + 1, STRIDE):
        for x in range(0, img_width - WINDOW_SIZE + 1, STRIDE):
            window_input = input_image.crop((x, y, x + WINDOW_SIZE, y + WINDOW_SIZE))
            window_golden = golden_image.crop((x, y, x + WINDOW_SIZE, y + WINDOW_SIZE))

            window_input_gray = np.array(window_input.convert('L'))
            window_golden_gray = np.array(window_golden.convert('L'))

            ssim_score, _ = ssim(window_golden_gray, window_input_gray, full=True)

            if ssim_score < SIMILARITY_THRESHOLD:
                patch_tensor = inference_transform(window_input).unsqueeze(0).to(device)
                predicted_idx, confidence = debug_top_predictions(patch_tensor, (x,y))

                if confidence > CLASSIFIER_CONFIDENCE_THRESHOLD:
                    detections.append({
                        'box': [x, y, x + WINDOW_SIZE, y + WINDOW_SIZE],
                        'label': class_names[predicted_idx],
                        'confidence': float(confidence)
                    })

    if not detections:
        return []

    # NMS
    boxes = torch.tensor([d['box'] for d in detections], dtype=torch.float32)
    scores = torch.tensor([d['confidence'] for d in detections], dtype=torch.float32)
    keep_indices = nms(boxes, scores, iou_threshold=0.05)
    return [detections[i] for i in keep_indices]

def draw_detections_on_image(image, detections):
    img_with_boxes = image.copy()
    draw = ImageDraw.Draw(img_with_boxes)

    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 32)
    except:
        font = ImageFont.load_default()

    unique_labels = list(set([d['label'] for d in detections]))
    colors = plt.cm.get_cmap('hsv', len(unique_labels) + 1)
    color_map = {label: tuple((np.array(colors(i)[:3]) * 255).astype(int))
                for i, label in enumerate(unique_labels)}

    for det in detections:
        box = det['box']
        label = det['label']
        confidence = det['confidence']
        color = color_map.get(label, (255, 50, 50))

        draw.rectangle(box, outline=color, width=5)

        text = f"{label} ({confidence:.2f})"
        text_bbox = draw.textbbox((0, 0), text, font=font)
        text_width = text_bbox[2] - text_bbox[0]
        text_height = text_bbox[3] - text_bbox[1]

        background_box = [box[0], box[1] - text_height - 5,
                         box[0] + text_width + 10, box[1]]
        draw.rectangle(background_box, fill=color)
        draw.text((box[0] + 5, box[1] - text_height - 5), text, fill="white", font=font)

    return img_with_boxes

🔍 Checkpoint keys: ['model_state_dict', 'class_names']
✅ 6 classes: ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']
✅ Model output shape: torch.Size([1, 6]) == [1, 6] ✓
✅ 10 golden images loaded


In [ ]:
test_image_path = "/content/drive/MyDrive/PCB_DATASET/images/Missing_hole/01_missing_hole_06.jpg"
test_image_name = os.path.basename(test_image_path)

print(f"\n--- Running pipeline on image: {test_image_name} ---")

try:
    input_image = Image.open(test_image_path).convert('RGB')

    golden_image_ref = find_best_match(input_image, golden_db)

    if golden_image_ref:
        anomalies = detect_anomalies_by_comparison(
            input_image,
            golden_image_ref,
            defect_classifier
        )

        result_image = draw_detections_on_image(input_image, anomalies)

        plt.figure(figsize=(20, 15))
        plt.imshow(result_image)
        plt.title(
            f"Detected Anomalies on '{test_image_name}' by Differential Comparison",
            fontsize=20
        )
        plt.axis('off')
        plt.show()
    else:
        print("No matching golden image found.")

except FileNotFoundError:
    print(f"ERROR: Test file '{test_image_path}' not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

STREAMLIT INTERFACE

---



In [ ]:
def run_inference_on_pil(input_image):
    start_time = time.time()

    print("🔍 Finding best golden match...")
    golden_image_ref = find_best_match(input_image, golden_db)
    if not golden_image_ref:
        print("⚠️ No golden reference found")
        return input_image, []

    print("🔍 Running differential detection...")
    anomalies = detect_anomalies_by_comparison(input_image, golden_image_ref, defect_classifier)
    result_image = draw_detections_on_image(input_image, anomalies)

    elapsed = time.time() - start_time
    print(f"✅ Detection complete: {len(anomalies)} defects in {elapsed:.2f}s")
    print(f"   Classes used: {class_names}")

    return result_image, anomalies

print("✅ PIPELINE READY - LABELS FIXED!")
print(f"Classes: {class_names}")
print("Run: streamlit run app.py")

✅ PIPELINE READY - LABELS FIXED!
Classes: ['missing_hole', 'mouse_bite', 'open_circuit', 'short', 'spur', 'spurious_copper']
Run: streamlit run app.py


SAVE OUTPUT IN FOLDER

In [ ]:
input_dir = "/content/drive/MyDrive/Images pcb/01_Short"
output_dir = "/content/drive/MyDrive/resnet50_model/Results/Short"
os.makedirs(output_dir, exist_ok=True)

for file_name in os.listdir(input_dir):
    if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):  # filter image files
        test_image_path = os.path.join(input_dir, file_name)
        print(f"\n--- Running pipeline on image: {file_name} ---")

        try:
            input_image = Image.open(test_image_path).convert('RGB')
            golden_image_ref = find_best_match(input_image, golden_db)
            if golden_image_ref:
                anomalies = detect_anomalies_by_comparison(
                    input_image,
                    golden_image_ref,
                    defect_classifier
                )

                result_image = draw_detections_on_image(input_image, anomalies)
                save_path = os.path.join(output_dir, f"inference_{file_name}")
                result_image.save(save_path)
                print(f"Saved inference result to: {save_path}")

            else:
                print(f"No matching golden image found for {file_name}.")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")


--- Running pipeline on image: 01_short_19.jpg ---
🔍 Window (2272, 0): Top-3 predictions:
  1. spur: 0.505
  2. mouse_bite: 0.276
  3. open_circuit: 0.100
🔍 Window (2304, 0): Top-3 predictions:
  1. spur: 0.522
  2. mouse_bite: 0.291
  3. open_circuit: 0.092
🔍 Window (256, 32): Top-3 predictions:
  1. spur: 0.550
  2. mouse_bite: 0.318
  3. open_circuit: 0.071
🔍 Window (288, 32): Top-3 predictions:
  1. spur: 0.545
  2. mouse_bite: 0.334
  3. open_circuit: 0.069
🔍 Window (448, 32): Top-3 predictions:
  1. spur: 0.522
  2. mouse_bite: 0.319
  3. open_circuit: 0.094
🔍 Window (480, 32): Top-3 predictions:
  1. spur: 0.517
  2. mouse_bite: 0.344
  3. open_circuit: 0.080
🔍 Window (512, 32): Top-3 predictions:
  1. spur: 0.473
  2. mouse_bite: 0.320
  3. open_circuit: 0.120
🔍 Window (672, 32): Top-3 predictions:
  1. spur: 0.519
  2. mouse_bite: 0.343
  3. open_circuit: 0.089
🔍 Window (704, 32): Top-3 predictions:
  1. spur: 0.540
  2. mouse_bite: 0.342
  3. open_circuit: 0.068
🔍 Window (73

/tmp/ipython-input-3909648523.py:151: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colors = plt.cm.get_cmap('hsv', len(unique_labels) + 1)


Streaming output truncated to the last 5000 lines.
  1. missing_hole: 0.680
  2. spur: 0.123
  3. spurious_copper: 0.064
🔍 Window (1184, 1024): Top-3 predictions:
  1. missing_hole: 0.848
  2. spur: 0.108
  3. mouse_bite: 0.016
🔍 Window (1216, 1024): Top-3 predictions:
  1. spur: 0.422
  2. mouse_bite: 0.237
  3. open_circuit: 0.146
🔍 Window (1248, 1024): Top-3 predictions:
  1. spur: 0.394
  2. mouse_bite: 0.245
  3. open_circuit: 0.186
🔍 Window (1280, 1024): Top-3 predictions:
  1. open_circuit: 0.443
  2. missing_hole: 0.236
  3. spur: 0.149
🔍 Window (1312, 1024): Top-3 predictions:
  1. missing_hole: 0.647
  2. open_circuit: 0.110
  3. spur: 0.106
🔍 Window (1344, 1024): Top-3 predictions:
  1. open_circuit: 0.357
  2. spur: 0.239
  3. short: 0.163
🔍 Window (1376, 1024): Top-3 predictions:
  1. missing_hole: 0.607
  2. open_circuit: 0.167
  3. short: 0.079
🔍 Window (1408, 1024): Top-3 predictions:
  1. missing_hole: 0.723
  2. spur: 0.109
  3. spurious_copper: 0.055
🔍 Window (1440, 



---



---



In [ ]:
#result_image, detections = run_inference_on_pil(input_image)


In [ ]:
# import torch
# from torchvision.ops import box_iou

# def compute_iou(boxA, boxB):
#     xA = max(boxA[0], boxB[0])
#     yA = max(boxA[1], boxB[1])
#     xB = min(boxA[2], boxB[2])
#     yB = min(boxA[3], boxB[3])

#     inter_w = max(0, xB - xA)
#     inter_h = max(0, yB - yA)
#     inter_area = inter_w * inter_h

#     areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
#     areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

#     union = areaA + areaB - inter_area
#     return inter_area / union if union > 0 else 0.0


# def print_iou_between_detections(detections, min_iou=0.05):
#     n = len(detections)
#     if n < 2:
#         print("Only one detection → no overlap to compute")
#         return

#     print("🔍 IoU between overlapping boxes:")
#     for i in range(n):
#         for j in range(i + 1, n):
#             if detections[i]['label'] == detections[j]['label']:
#                 iou = compute_iou(
#                     detections[i]['box'],
#                     detections[j]['box']
#                 )
#                 if iou >= min_iou:
#                     print(
#                         f"Box {i} ↔ Box {j} "
#                         f"[{detections[i]['label']}]: IoU = {iou:.3f}"
#                     )
# print_iou_between_detections(detections)
